In [ ]:
#------------------Import & Check Dataset------------------------------------

In [119]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv("2019 Winter Data Science Intern Challenge Data Set - Sheet1.csv")
df.head()

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,1,53,746,224,2,cash,2017/3/13 12:36
1,2,92,925,90,1,cash,2017/3/3 17:38
2,3,44,861,144,1,cash,2017/3/14 4:23
3,4,18,935,156,1,credit_card,2017/3/26 12:43
4,5,18,883,156,1,credit_card,2017/3/1 4:35


In [120]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   order_id        5000 non-null   int64 
 1   shop_id         5000 non-null   int64 
 2   user_id         5000 non-null   int64 
 3   order_amount    5000 non-null   int64 
 4   total_items     5000 non-null   int64 
 5   payment_method  5000 non-null   object
 6   created_at      5000 non-null   object
dtypes: int64(5), object(2)
memory usage: 273.6+ KB


In [121]:
df.nunique()

order_id          5000
shop_id            100
user_id            301
order_amount       258
total_items          8
payment_method       3
created_at        4702
dtype: int64

In [122]:
df['created_at']=pd.to_datetime(df['created_at'])
df['date'] = df.created_at.dt.date
df['hour'] = df.created_at.dt.hour
df.head()

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at,date,hour
0,1,53,746,224,2,cash,2017-03-13 12:36:00,2017-03-13,12
1,2,92,925,90,1,cash,2017-03-03 17:38:00,2017-03-03,17
2,3,44,861,144,1,cash,2017-03-14 04:23:00,2017-03-14,4
3,4,18,935,156,1,credit_card,2017-03-26 12:43:00,2017-03-26,12
4,5,18,883,156,1,credit_card,2017-03-01 04:35:00,2017-03-01,4


In [123]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   order_id        5000 non-null   int64         
 1   shop_id         5000 non-null   int64         
 2   user_id         5000 non-null   int64         
 3   order_amount    5000 non-null   int64         
 4   total_items     5000 non-null   int64         
 5   payment_method  5000 non-null   object        
 6   created_at      5000 non-null   datetime64[ns]
 7   date            5000 non-null   object        
 8   hour            5000 non-null   int64         
dtypes: datetime64[ns](1), int64(6), object(2)
memory usage: 351.7+ KB


In [124]:
#------------------ Analysis------------------------------------------------

In [125]:
#------------------Customers--------------------------------------------------

In [126]:
df_behavior=df.groupby(by='user_id')
df_behavior=df.user_id.value_counts()
df_behavior=pd.DataFrame(df_behavior)
df_behavior=df_behavior.reset_index()
df_behavior.head()

,index,user_id
0,718,28
1,868,27
2,791,26
3,847,26
4,787,25


In [127]:
##Repurchase Rate
repurchase_user = df_behavior[df_behavior['user_id']>=2].shape[0]

In [128]:
repurchase_user#repurchase_rate = 100%

301

In [129]:
#Customer Classification (RFM) (Recency，Frequency，Monetary）
df1=df.groupby(by='user_id')

RF = pd.DataFrame(index=df1.groups.keys(), columns=['Recency', 'Frequency'])
RF['Frequency']= df1.user_id.count()
RF['Last Purchase Time'] = df1.created_at.max()
RF['Recency']= (pd.to_datetime('2017-3-31')-RF['Last Purchase Time']).dt.days #Number of days for each elements

In [130]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   order_id        5000 non-null   int64         
 1   shop_id         5000 non-null   int64         
 2   user_id         5000 non-null   int64         
 3   order_amount    5000 non-null   int64         
 4   total_items     5000 non-null   int64         
 5   payment_method  5000 non-null   object        
 6   created_at      5000 non-null   datetime64[ns]
 7   date            5000 non-null   object        
 8   hour            5000 non-null   int64         
dtypes: datetime64[ns](1), int64(6), object(2)
memory usage: 351.7+ KB


In [131]:
RF.head()

,Recency,Frequency,Last Purchase Time
607,2,17,2017-03-28 04:00:00
700,0,16,2017-03-30 23:55:00
701,3,13,2017-03-27 13:54:00
702,1,13,2017-03-29 07:10:00
703,1,16,2017-03-29 10:25:00


In [132]:
RF[['Recency','Frequency']].describe().round(2)

,Recency,Frequency
count,301.00,301.00
mean,1.36,16.61
std,1.81,3.97
min,0.00,7.00
25%,0.00,14.00
50%,1.00,16.00
75%,2.00,19.00
max,12.00,28.00


In [133]:
def R_score(x):
    if 0 <= x <= 2:
        return 3
    elif 3 <= x <= 6:
        return 2
    elif 7 <= x <= 10:
        return 1
    else:
        return 0

In [134]:
def F_score(x):
    if 7 <= x <= 11:
        return 0
    elif 12 <= x <= 16:
        return 1
    elif 17<= x <= 21:
        return 2
    else:
        return 3

In [135]:
RF['R_score'] = RF.Recency.map(R_score)
RF['F_score'] = RF.Frequency.map(F_score)

In [136]:
def customer_classfication(x):
    R_score, F_score = x
    if 2 <= R_score <= 3  and 2<= F_score <= 3:
        return "Loyal Customer"
    elif 2 <= R_score <= 3 and 0<= F_score <= 1:
        return "Active Customer"
    elif 0 <= R_score <= 1  and 2<= F_score <= 3:
        return "Prospect Customer"
    elif 0 <= R_score <= 1 and 0<= F_score <= 1:
        return "Suspect Customer"
    else:
        return None

In [137]:
RF['Classification']=RF[['R_score','F_score']].apply(customer_classfication,axis=1)

In [138]:
RF.Classification.value_counts()

Active Customer      154
Loyal Customer       140
Suspect Customer       6
Prospect Customer      1
Name: Classification, dtype: int64

In [139]:
import plotly.graph_objects as go

labels = ["Active Customers","Loyal Customers" ,"Suspect Customers","Prospect Customers"]
values = [154,140,6,1]

# Create subplots: use 'domain' type for Pie subplot
fig = go.Figure(data=[go.Pie(labels=labels, values=values)])

# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.5, hoverinfo="label+percent")

fig.update_layout(
    title_text="Customer Classification",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='Total: 301', x=0.5, y=0.5, font_size=20, showarrow=False)],
    font=dict(size=20))
    
fig.show()

In [140]:
##Customer Lifetime Value

In [141]:
df['unit_price']=df['order_amount']/df['total_items']

In [142]:
CLV_df=df[['user_id','created_at','date','hour','order_amount','unit_price','total_items']]
CLV_df['count']=1

/tmp/ipykernel_18357/204520890.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [143]:
CLV_df_group=CLV_df.groupby('user_id').agg({'created_at': lambda date: (date.max() - date.min()).days,
                                            'count': lambda  quant: quant.sum(),
                                            'total_items': lambda quant: quant.sum(),
                                            'order_amount': lambda price: price.sum()})

In [144]:
CLV_df_group=CLV_df_group.rename(columns={'created_at': 'Duration', 'count': 'Frequency','order_amount':'Total Order Amount','total_items':'Total Quantity'})

In [145]:
#AOV
CLV_df_group['AOV']=(CLV_df_group['Total Order Amount']/CLV_df_group['Frequency']).round(3)

In [146]:
#Purchase_frequency
purchase_frequency=sum(CLV_df_group['Frequency'])/CLV_df_group.shape[0]
purchase_frequency

16.611295681063122

In [147]:
#Repeat Rate >10
repeat_rate=CLV_df_group[CLV_df_group.Frequency > 10].shape[0]/CLV_df_group.shape[0]
repeat_rate

0.9568106312292359

In [148]:
#Churn Rate
churn_rate=1-repeat_rate

In [149]:
purchase_frequency,repeat_rate,churn_rate

(16.611295681063122, 0.9568106312292359, 0.04318936877076407)

In [150]:
# Profit Margin
CLV_df_group['Profit_Margin']=CLV_df_group['Total Order Amount']*0.05

In [151]:
# Customer Value
CLV_df_group['Customer_Value']=(CLV_df_group['AOV']*purchase_frequency)/churn_rate

In [152]:
#Customer Lifetime Value
CLV_df_group['Customer Lifetime Value']=(CLV_df_group['Customer_Value']*CLV_df_group['Profit_Margin']).round(3)

In [153]:
CLV_df_group.tail(n=30)

,Duration,Frequency,Total Quantity,Total Order Amount,AOV,Profit_Margin,Customer_Value,Customer Lifetime Value
user_id,,,,,,,,
970,26,20,42,31449,1572.450,1572.45,6.047885e+05,9.509996e+08
971,25,15,30,4560,304.000,228.00,1.169231e+05,2.665846e+07
972,24,17,36,5171,304.176,258.55,1.169908e+05,3.024796e+07
973,23,11,24,3062,278.364,153.10,1.070631e+05,1.639136e+07
974,27,18,36,5640,313.333,282.00,1.205127e+05,3.398458e+07
975,28,23,44,6959,302.565,347.95,1.163712e+05,4.049134e+07
976,23,14,30,4464,318.857,223.20,1.226373e+05,2.737265e+07
977,25,13,28,4066,312.769,203.30,1.202958e+05,2.445613e+07
978,26,21,35,5140,244.762,257.00,9.413923e+04,2.419378e+07


In [154]:
CLV_df_group = CLV_df_group.sort_values(by='Customer Lifetime Value',ascending = False)

In [158]:
pd.set_option('display.float_format',lambda x : '%.2f' % x)
CLV_df_group.head(n=10)

,Duration,Frequency,Total Quantity,Total Order Amount,AOV,Profit_Margin,Customer_Value,Customer Lifetime Value
user_id,,,,,,,,
607,26,17,34000,11968000,704000.00,598400.00,270769230.77,162028307692307.88
878,25,11,24,156936,14266.91,7846.80,5487272.69,43057531362.00
834,26,18,38,108342,6019.00,5417.10,2315000.00,12540586500.00
766,23,10,23,80076,8007.60,4003.80,3079846.15,12331088030.77
915,29,14,29,80992,5785.14,4049.60,2225055.00,9010582728.00
962,26,14,24,80408,5743.43,4020.40,2209011.15,8881108442.92
775,24,16,28,81114,5069.62,4055.70,1949855.77,7908030043.27
817,25,16,30,80988,5061.75,4049.40,1946826.92,7883480942.31
928,26,16,28,80668,5041.75,4033.40,1939134.62,7821305557.69


In [159]:
CLV_df_group.tail(n=10)

,Duration,Frequency,Total Quantity,Total Order Amount,AOV,Profit_Margin,Customer_Value,Customer Lifetime Value
user_id,,,,,,,,
998,23,9,18,2620,291.11,131.00,111965.77,14667515.77
751,15,11,19,2880,261.82,144.00,100699.23,14500689.23
722,28,13,21,3130,240.77,156.50,92603.46,14492441.73
882,27,11,18,2854,259.45,142.70,99790.38,14240087.88
827,27,14,21,3171,226.50,158.55,87115.38,13812144.23
892,27,13,20,2955,227.31,147.75,87426.15,12917214.23
806,21,10,19,2572,257.20,128.60,98923.08,12721507.69
719,29,9,15,2314,257.11,115.70,98888.85,11441439.50
902,27,9,14,2108,234.22,105.40,90085.38,9494999.54
